In [4]:
import cv2 as cv
import numpy as np
from matplotlib import cm
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
 
 
def read_image(path, color_code=cv.IMREAD_GRAYSCALE):
    """
    read a picture from path
    :type path: string
    :param path: the location of a picture
    :type color_code: opencv color code
    :param color_code: which type the image should be read, cv.IMREAD_GRAYSCALE as default
    :return: the picture read from the path, None if there is an error
    """
    return cv.imread(path, color_code)
  
def resize_img(img, width=800):
    """
    resize image
    :type img: image
    :param img: input image
    :type width: int
    :param width: width after resize,800 as default
    :return: image after resize
    """
    return cv.resize(img, (width, int(width * img.shape[0] / img.shape[1])))
 
def med_blur(img, ksize=3, flag=False):
    """
    Median filter for input image
    :param img: input image
    :param ksize: size of filter
    :return: image after median filter
    """
 
    if img.dtype is not np.uint8:
        img = img.astype(np.uint8)
 
    new_img = cv.medianBlur(img, ksize)
    if flag:
        plt.rcParams['font.sans-serif'] = ['SimHei']  # 正确打印中文
        plt.subplot(2, 2, 1)
        plt.imshow(img, cmap="gray")
        plt.title("原图")
        plt.subplot(2, 2, 2)
        plt.hist(img)
        plt.title("原图直方图")
        plt.subplot(2, 2, 3)
        plt.imshow(new_img, cmap="gray")
        plt.title("中值滤波结果")
        plt.subplot(2, 2, 4)
        plt.hist(new_img)
        plt.title("中值滤波结果直方图")
        plt.show()
    return new_img
 
 
def gauss_blur(img, ksize=[3, 3]):
    cv.GaussianBlur(img, ksize=ksize)
 
 


In [5]:
# coding:utf-8
import time
import cv2 as cv
from matplotlib import pyplot as plt
import numpy as np
import os
import cv2
import os
import numpy as np
from skimage.color import rgb2hsv
from skimage import filters

def enhance_contrast(image):
    # 對比增強
    return cv2.equalizeHist(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))

def rgb_to_hsv(image):
    # RGB轉HSI/HSV
    hsv_image = rgb2hsv(image)
    intensity = hsv_image[2:, :, 2]  # 提取飽和度通道
    return intensity

def calculate_otsu_threshold(image):
    # 計算 Otsu 閾值
    return filters.threshold_otsu(image)

def apply_joint_threshold(image, otsu_thresh):
    # 計算聯合閾值
    _, binary_global = cv2.threshold(image, otsu_thresh, 255, cv2.THRESH_BINARY)
    return binary_global

def canny_edge_detection(image):
    # Canny 邊緣檢測
    return cv2.Canny(image, 100, 200)

def morphological_operations(image):
    # 形態學操作
    #kernel = np.ones((5, 5), np.uint8)
    #closed_image = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
    fat_image = cv.dilate(image, None, iterations=2)
    return fat_image

def process_image(file_path, output_dir):
    """
    Process a single image using RegionGrow.
    :param file_path: Path to the input image.
    :param output_dir: Directory to save the processed image.
    """
    # 读取图片
    origin = read_image(file_path, color_code=cv.IMREAD_ANYCOLOR)  # Read color image  
    # 裁剪图片
    img = resize_img(origin)
       
    # 步驟 1: 對比增強
    enhanced_image = enhance_contrast(img)

    # 步驟 2: RGB 轉 HSI/HSV
    intensity = rgb_to_hsv(img)

    # 步驟 3: 計算 Otsu 閾值
    otsu_thresh = calculate_otsu_threshold(intensity)

    # 步驟 4: 計算聯合閾值
    joint_thresholded = apply_joint_threshold(enhanced_image, otsu_thresh)

    # 步驟 5: Canny 邊緣檢測
    edges = canny_edge_detection(joint_thresholded)

    # 步驟 6: 形態學操作
    final_image = morphological_operations(edges)
    
    # 中值滤波
    final_image = med_blur(final_image, ksize=3, flag=False)   

    # Save the processed image with the same name in the output directory
    output_path = os.path.join(output_dir, os.path.basename(file_path))
    plt.imsave(output_path, final_image, cmap='gray')
    #print(f"Saved processed image to {output_path}")

if __name__ == "__main__":
    input_dir = "/home/yuchi/Canny_Detection/crackforest/Images/"  # Input directory
    output_dir = "/home/yuchi/Canny_Detection/crackforest/Output_1/"  # Output directory
    os.makedirs(output_dir, exist_ok=True)  # Create output directory if it doesn't exist

    # Process each image in the input directory
    for file_name in os.listdir(input_dir):
        file_path = os.path.join(input_dir, file_name)
        if os.path.isfile(file_path) and file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            try:
                process_image(file_path, output_dir)
            except Exception as e:
                print(f"Error processing {file_name}: {e}")


In [6]:

import os
import cv2
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

def read_images_with_matching(folder, prefix, file_extension):
    """读取指定资料夹中指定格式的图片，并匹配文件名前缀"""
    image_files = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith(file_extension)]
    images = {}
    for file in image_files:
        # 获取不含后缀的文件名，并去掉可能的 '_label'
        image_name = os.path.splitext(os.path.basename(file))[0].replace('_label', '')
        image = cv2.imread(file, cv2.IMREAD_GRAYSCALE)  # 读取灰度图像
        if image is not None:
            images[image_name] = image
        else:
            print(f"Warning: Could not read {file}")
    return images

def calculate_metrics(true_images, predicted_images, threshold=128):
    """计算 Precision, Recall 和 F-Measure"""
    precision_list, recall_list, f1_list = [], [], []
    
    for name, true_image in true_images.items():
        if name in predicted_images:
            pred_image = predicted_images[name]
            
            # 确保两张图片尺寸一致
            if true_image.shape != pred_image.shape:
                print(f"Skipping {name}: Shape mismatch ({true_image.shape} vs {pred_image.shape})")
                continue
            
            # 二值化处理
            true_binary = (true_image > threshold).astype(np.uint8).flatten()
            pred_binary = (pred_image > threshold).astype(np.uint8).flatten()
            
            # 计算指标
            precision = precision_score(true_binary, pred_binary, zero_division=0)
            recall = recall_score(true_binary, pred_binary, zero_division=0)
            f1 = f1_score(true_binary, pred_binary, zero_division=0)
            
            precision_list.append(precision)
            recall_list.append(recall)
            f1_list.append(f1)
        else:
            print(f"Warning: Missing matching image for {name} in predicted folder")
    
    # 计算平均指标
    avg_precision = np.mean(precision_list) if precision_list else 0
    avg_recall = np.mean(recall_list) if recall_list else 0
    avg_f1 = np.mean(f1_list) if f1_list else 0
    
    return avg_precision, avg_recall, avg_f1

def main(jpg_folder, png_folder, threshold=128):
    """主函数"""
    # 读取两个文件夹中的图片
    print("Reading .jpg images...")
    true_images = read_images_with_matching(jpg_folder, '', '.jpg')
    
    print("Reading .PNG images...")
    predicted_images = read_images_with_matching(png_folder, '_label', '.PNG')
    
    # 计算 Precision, Recall 和 F-Measure
    print("Calculating metrics...")
    precision, recall, f1 = calculate_metrics(true_images, predicted_images, threshold)
    
    # 输出结果
    print("\nResults:")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F-Measure: {f1:.4f}")

# 示例运行
if __name__ == "__main__":
    jpg_folder = "/home/yuchi/Canny_Detection/crackforest/Output_1"  # 替换为您的 .jpg 图片资料夹路径
    png_folder = "/home/yuchi/Canny_Detection/crackforest/Masks_copy"  # 替换为您的 .PNG 图片资料夹路径
    main(jpg_folder, png_folder, threshold=128)


Reading .jpg images...
Reading .PNG images...
Calculating metrics...

Results:
Precision: 0.4171
Recall: 0.4276
F-Measure: 0.3999
